In [3]:
import math
from typing import Optional, List
import torch
from torch import nn
from labml import tracker

In [5]:
class PrepareForMultiHeadAttention(nn.Module): # nn.Module
    def __init__(self, d_model, heads, d_k, bias): 
        '''
            var 'heads' is the number of heads
            var 'd_model' is the number of features in q, k, v
            var 'd_k' is the demention of each head
            var bias is ?
        '''
        super().__init__
        self.linear = nn.Linear(d_model, heads*d_k, bias = bias)
        self.heads = heads
        self.d_k = d_k

    def forward(self, x: torch.Tensor):
        '''
            var x is the vector of all
        '''
        head_shape = x.shape[:-1] #??? #输入的形状为[seq_len, batch_size, d_model] 或[batch_size, d_model] 。我们对最后一维应用线性变换，并将其分为多个头。
        x = self.linear(x) #???
        x = x.view(*head_shape, self.heads, self.d_k) #???
        return x

In [19]:
x = [[1,2,3,4,5],[6,7,8,9,10],[11,12,13,14,15]]
x = torch.tensor(x)

print(x.shape)
d_model = 3
heads = 3
d_k = 1
bias = 1
linear = nn.Linear(d_model, heads*d_k, bias = bias)
print(x)
x = linear(x)
print(x)


torch.Size([3, 5])
tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10],
        [11, 12, 13, 14, 15]])


RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float

In [16]:
print(*9)

TypeError: print() argument after * must be an iterable, not int